# Importing from the COVID Tracking Project

This script pulls data from the API provided by the [COVID Tracking Project](https://covidtracking.com/). They're collecting data from 50 US states, the District of Columbia, and five U.S. territories to provide the most comprehensive testing data. They attempt to include positive and negative results, pending tests and total people tested for each state or district currently reporting that data.

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
import pycountry

In [2]:
# papermill parameters
# output_folder = '../output/'

In [2]:
raw_response = requests.get("https://covidtracking.com/api/v1/states/daily.json").text
raw_data = pd.DataFrame.from_dict(json.loads(raw_response))

### Data Quality
1. Replace empty values with zero
2. Convert "date" int column to "Date" datetime column
4. Rename columns in order to match with other source
5. Drop unnecessary columns
6. Add "Country/Region" column, since the source contains data from US states, it can be hardcoded

In [3]:
data = raw_data.fillna(0)
data['Date'] = pd.to_datetime(data['date'].astype(str), format='%Y%m%d')
data = data.rename(
    columns={
        "state": "ISO3166-2",
        "totalTestResults": "Total"
})
data = data.drop(labels=['dateChecked', "date","hash"], axis='columns')
data['Country/Region'] = "United States"
data['ISO3166-1'] = "US"


In [4]:
states = {k.code.replace("US-", ""): k.name for k in pycountry.subdivisions.get(country_code="US")}

In [5]:
data["Province/State"] = data["ISO3166-2"].apply(lambda x: states[x])

## Sorting data by Province/State before calculating the daily differences

In [6]:
data = data.sort_values(by=['Province/State'] + ['Date'], ascending=True)

## Add `Last_Update_Date`

In [7]:
data["Last_Update_Date"] = datetime.datetime.utcnow()
data['Last_Reported_Flag'] = data['Date'].max() == data['Date']

## Export to CSV

The example JSON reponse is:

```
[{"date":20200411,"state":"AK","positive":257,"negative":7475,"pending":null,"hospitalizedCurrently":null,"hospitalizedCumulative":31,"inIcuCurrently":null,"inIcuCumulative":null,"onVentilatorCurrently":null,"onVentilatorCumulative":null,"recovered":63,"hash":"a8d36e9ce19edaeaac989881abf96fc74196efba","dateChecked":"2020-04-11T20:00:00Z","death":8,"hospitalized":31,"total":7732,"totalTestResults":7732,"posNeg":7732,"fips":"02","deathIncrease":1,"hospitalizedIncrease":3,"negativeIncrease":289,"positiveIncrease":11,"totalTestResultsIncrease":300}
```

In [8]:
cal_tests = data[data['Province/State'] == 'California']
cal_tests.to_csv('cal_tests.csv', columns=['Province/State', 'Date',
                               'positive', 'positiveIncrease',
                               'negative', 'negativeIncrease', 
                               'death', 'deathIncrease',
                               'total', 'totalTestResultsIncrease' ], index=False)

In [9]:
non_ca = data[(data['Province/State'] != 'California')  & (data['Date'] >= '2020-01-01')]
non_ca.to_csv('non_ca.csv', columns=['Province/State', 'Date',
                               'positive', 'positiveIncrease',
                               'negative', 'negativeIncrease', 
                               'total', 'totalTestResultsIncrease', 'positiveTestsViral', 'negativeTestsViral', 'totalTestsViral', 
                                     'death', 'deathIncrease'], index=False)

In [10]:

cal_tests.to_csv("covid_testing.csv", columns=['Province/State', 'Date',
                               'positive', 'positiveIncrease',
                               'negative', 'negativeIncrease', 
                               'death', 'deathIncrease',
                               'total', 'totalTestResultsIncrease' ], index=False)

In [11]:
data.to_csv("us_covid_testing.csv", columns=['Province/State', 'Date',
                               'positive', 'positiveIncrease',
                               'negative', 'negativeIncrease', 
                               'death', 'deathIncrease',
                               'total', 'totalTestResultsIncrease' ], index=False)